In [1]:
#Sentiment Analysis Tsla using Beautiful Soup

In [7]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
import requests

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("NEWS_API_KEY")

In [10]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key)
print(api_key)

1ee42792279e4bb38c96b682f9029b47


In [11]:
# Fetch the TSLA news articles
tesla_headlines = newsapi.get_everything(
    q="Tesla",
    language="en",
    page_size=100,
    sort_by="relevancy"
)
tesla_headlines

{'status': 'ok',
 'totalResults': 7675,
 'articles': [{'source': {'id': 'engadget', 'name': 'Engadget'},
   'author': 'Igor Bonifacic',
   'title': "Tesla's deliveries increased despite supply shortages and plant closures",
   'description': 'Tesla delivered 310,048 vehicles over the first three months of 2022, the automaker announced\r\n on Saturday. “This was an exceptionally difficult quarter due to supply chain interruptions and China Zero-Covid policy,” Musk said\r\n on Twitter shortly after Tesl…',
   'url': 'https://www.engadget.com/tesla-q-1-2022-deliveries-214141313.html',
   'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2022-03/733d9be0-b2cd-11ec-b7f9-382caaedc1ae',
   'publishedAt': '2022-04-02T21:41:41Z',
   'content': 'Tesla delivered 310,048 vehicles over the first three months of 2022, the automaker announced\r\n on Saturday. This was an exceptionally difficult quarter due to supply chain interruptions and China Ze… [+920 chars]'},
  {'source': {'id': 'enga

In [14]:
# Create the Tesla sentiment scores DataFrame
tesla_sentiments = []

for article in tesla_headlines["articles"]:
    try:
        text = article["content"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        tesla_sentiments.append({
            "text": text,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
tesla_df = pd.DataFrame(tesla_sentiments)   
tesla_df

,text,compound,positive,negative,neutral
0,"Tesla delivered 310,048 vehicles over the firs...",-0.6697,0.000,0.155,0.845
1,Tesla has raised the prices of its electric ve...,0.0000,0.000,0.000,1.000
2,Tesla has delivered the first 30 Model Y cars ...,0.6808,0.157,0.000,0.843
3,Tesla will finally start selling the Cybertruc...,0.4019,0.074,0.000,0.926
4,Tesla and SpaceX CEO Elon Musk now owns 9.2 pe...,0.2960,0.062,0.000,0.937
...,...,...,...,...,...
95,"Tesla's Model 3, introduced as a mass-market e...",0.0935,0.040,0.000,0.960
96,For several years now we’ve been following [Ja...,0.2263,0.051,0.000,0.949
97,Tesla vehicles have a charging port that is un...,0.2732,0.058,0.000,0.942
98,"NEW YORK, April 4 (Reuters) - Some well-timed ...",0.2023,0.050,0.000,0.950


In [17]:
# Creating a binary response for the tsla stock
tesla_df['comp_score'] = tesla_df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

tesla_df.head(20)

,text,compound,positive,negative,neutral,comp_score
0,"Tesla delivered 310,048 vehicles over the firs...",-0.6697,0.000,0.155,0.845,neg
1,Tesla has raised the prices of its electric ve...,0.0000,0.000,0.000,1.000,pos
2,Tesla has delivered the first 30 Model Y cars ...,0.6808,0.157,0.000,0.843,pos
3,Tesla will finally start selling the Cybertruc...,0.4019,0.074,0.000,0.926,pos
4,Tesla and SpaceX CEO Elon Musk now owns 9.2 pe...,0.2960,0.062,0.000,0.937,pos
5,Its set to open later this year\r\nIf you buy ...,-0.4404,0.000,0.081,0.919,neg
6,Morning Express with Robin Meade,0.0000,0.000,0.000,1.000,pos
7,"Just when I thought the NFT chaos had calmed, ...",-0.2732,0.070,0.099,0.831,neg
8,U.S. stock index futures rose on Monday as inv...,0.7430,0.160,0.000,0.840,pos
9,Its time to talk about all of those other new ...,0.4215,0.076,0.000,0.924,pos
